In [ ]:
# SETUP

# Install our additional project requirements
%pip install --quiet --no-index --find-links=/kaggle/input/mammography-move/wheels hydra-core dicomsdl

%env PYTHONPATH /kaggle/input/mammography-move/kaggle/

In [ ]:
from sys import path
path.append("/kaggle/input/mammography-move/kaggle/")

# Generate metadata
from mammography.src.data.metadata import get_breast_metadata
import pandas as pd

image_meta = pd.read_csv("/kaggle/input/rsna-breast-cancer-detection/test.csv")
breast_meta = get_breast_metadata(image_meta)

breast_meta.to_json("metadata.json", orient="records")

# Generate random predictions for testing
# import numpy as np
# breast_meta["cancer"] = np.random.randint(low=0, high=2, size=len(breast_meta))
# breast_meta[["prediction_id", "cancer"]].to_csv("submission.csv", index=False)

In [ ]:
# Submit
!mkdir -p /kaggle/working/torch/hub/
!cp -r /kaggle/input/mammography-move/torch/hub/ /kaggle/working/torch/hub/
%env TORCH_HOME /kaggle/working/torch/hub/
!python -m mammography.src.submit \
    trainer.accelerator=gpu \
    ckpt_path=/kaggle/input/mammography-move/checkpoint.ckpt \
    datamodule.image_dir=/kaggle/input/rsna-breast-cancer-detection/test_images \
    datamodule.metadata_paths.predict=metadata.json